# Collecting English data using youtube api

AWS를 통한 thumbnail text 분석을 위해 영어로 된 정보를 수집한다.
1. collect a vedio data list with English keyword
2. collect view count
3. make csv file

In [1]:
import pandas as pd
from googleapiclient.discovery import build
import config
from urllib.request import urlopen

In [2]:
youtube = build('youtube', 'v3', developerKey = config.API_KEY)

In [3]:
#첫 수집 시
'''
df = pd.DataFrame(
        columns = ['title', 'video_id', 'channel_name', 'channel_id', 'thumbnail', 'publish_time', 'view', 'thumbnail_text'])
df
'''

"\ndf = pd.DataFrame(\n        columns = ['title', 'video_id', 'channel_name', 'channel_id', 'thumbnail', 'publish_time', 'view', 'thumbnail_text'])\ndf\n"

In [3]:
data_original = pd.read_csv("./english_data_including_thumbnailtext.csv")
df = pd.DataFrame(data_original)
df

,title,video_id,channel_name,channel_id,thumbnail,publish_time,view,thumbnail_text
0,VLOG: МЫ ТАК ПОДУМАЛИ! 25.05.21,kaibm6LSmb4,Katy LifeVlog,UCpm6vCgiehSKHtMFFS5ukiA,https://i.ytimg.com/vi/kaibm6LSmb4/mqdefault.jpg,2021-05-26T02:30:05Z,101580,"{'TextDetections': [{'DetectedText': 'Sweets',..."
1,RC Car with Train Horns on Golf Course,zYgAf2eu8sQ,Vlog Creations,UCR0VLWitB2xM4q7tjkoJUPw,https://i.ytimg.com/vi/zYgAf2eu8sQ/mqdefault.jpg,2021-05-24T16:00:21Z,1147867,{'TextDetections': [{'DetectedText': 'INSURANC...
2,Brought my FATHER&#39;s PILLOW to sleep | Ss V...,OtACAGFKpn8,Sambhavna Seth Entertainment,UChZ0JmGgyOWSU-1iXxPhsVA,https://i.ytimg.com/vi/OtACAGFKpn8/mqdefault.jpg,2021-05-26T06:25:15Z,429219,"{'TextDetections': [{'DetectedText': 'for', 'T..."
3,6AM PRODUCTIVE online school morning routine/v...,2_q4T1abM8Q,Kristy Bae,UCiLSb-cS_0pkjIYsm9oOB0w,https://i.ytimg.com/vi/2_q4T1abM8Q/mqdefault.jpg,2021-04-29T14:00:02Z,607709,"{'TextDetections': [{'DetectedText': 'I', 'Typ..."
4,My Funniest Vlog Ever 😂 Ayachi Thakur Vlogs,tHD5ajeSxKQ,Ayachi Thakur Vlogs,UC8RXP2epKG3uiPO5QYQyGqg,https://i.ytimg.com/vi/tHD5ajeSxKQ/mqdefault.jpg,2021-05-26T01:30:07Z,46413,"{'TextDetections': [], 'TextModelVersion': '3...."
...,...,...,...,...,...,...,...,...
8020,SI YO FUERA MADRE | Lyna Vlogs,HG6Qkqv5eBQ,Lyna Vlogs,UCRtkuS3Wz0hYgfJjpvEEP4Q,https://i.ytimg.com/vi/HG6Qkqv5eBQ/default.jpg,2017-10-08T22:39:42Z,4884575,not-yet
8021,| DHARAN | Dasain Vlog 2017 | Bhedetar,ZqPTPoyr2M4,Sisan Baniya,UC6SEUWgSEUOTKdwdRV7IRhA,https://i.ytimg.com/vi/ZqPTPoyr2M4/default.jpg,2017-10-09T06:36:39Z,296289,not-yet
8022,VLOG! ALA E TRUMP!,sA3Z_aHJrh8,MaxINFINITE,UCcLZdjaJEC_6RqfM3PhefIw,https://i.ytimg.com/vi/sA3Z_aHJrh8/default.jpg,2017-10-07T08:32:58Z,491894,not-yet
8023,VLOG #78: Hướng dẫn đi máy bay cho người đi lầ...,cwlD15TESUI,Yêu Máy Bay,UC6iYLWyzdYKMpnWbplhBrxg,https://i.ytimg.com/vi/cwlD15TESUI/default.jpg,2017-10-03T10:07:44Z,512585,not-yet


## 1. Collect a vedio data list with English keyword 
- title
- vedioid
- channelname
- channelid
- thumbnailurl
- publish time

In [5]:
#들어온 날짜의 바로 이전달 1일 계산
def calcuate_next_publish_time(time):
    if(time[5:7]=='01'):
        return '%04d-12-01T00:00:00Z'%(int(time[:4])-1)
    else:
        return time[:5]+'%02d-01T00:00:00Z'%(int(time[5:7])-1)

In [5]:
start_time = 'today'
for index in range(len(df)):
    if(df['publish_time'][index]!='not-yet'):
        start_time=df['publish_time'][index]
    else:
        break
start_time = start_time[:8]+'01T00:00:00Z'
print(start_time)

2018-10-01T00:00:00Z


In [6]:
start_time = calcuate_next_publish_time(start_time)
print(start_time)

2018-09-01T00:00:00Z


In [7]:
def get_youtube_vedio_search_list(keyword, start_time):    
    page_token = ''
    while(len(df)<8000):
        search_response = youtube.search().list(
            q = keyword,
            type='video',
            part = 'snippet, id',
            maxResults = 50,
            regionCode = 'US',
            pageToken = page_token,
            publishedBefore = start_time,
            publishedAfter = calcuate_next_publish_time(start_time)
        ).execute()

        new_data = {'title':'not-yet', 'video_id':'not-yet', 'channel_name':'not-yet', 'channel_id':'not-yet', 'thumbnail':'not-yet', 'publish_time':'not-yet', 'view':-1, 'thumbnail_text':'not-yet'}

        for item in search_response['items']:
            new_data['title'] = item['snippet']['title']
            new_data['video_id'] = item['id']['videoId']
            new_data['channel_name'] = item['snippet']['channelTitle']
            new_data['channel_id'] = item['snippet']['channelId']
            new_data['thumbnail'] = item['snippet']['thumbnails']['default']['url']
            new_data['publish_time'] = item['snippet']['publishTime']
            df.loc[len(df)] = new_data

        if ('nextPageToken' in search_response):
            page_token = search_response['nextPageToken']
        else:
            print('count : ', len(df))
            start_time = calcuate_next_publish_time(start_time)
            page_token=''

In [8]:
get_youtube_vedio_search_list('vlog', start_time)
print('unique data num: ' + str(len(df)))

count :  4865
count :  5375
count :  5624
count :  5865
count :  6114
count :  6628
count :  6826
count :  7086
count :  7591
count :  7836
unique data num: 8036


In [1]:
'''
#url open 불가한 영상 삭제
#df['thumbnail_byte']='not-yet'

print(len(df))
start_index = len(df)
for index in range(len(df)):
    if(df['thumbnail_text'][index]=='not-yet'):
        start_index=index
        break
print('start_index:', start_index)

for index in range(start_index, len(df)):
    try:
        df['thumbnail_text'][index] = urlopen(df['thumbnail'][index]).read()
        print(index)
    except:
        df.drop(index, inplace=True)
        print('drop', index)

print(len(df))
'''

"\n#url open 불가한 영상 삭제\n#df['thumbnail_byte']='not-yet'\n\nprint(len(df))\nstart_index = len(df)\nfor index in range(len(df)):\n    if(df['thumbnail_text'][index]=='not-yet'):\n        start_index=index\n        break\nprint('start_index:', start_index)\n\nfor index in range(start_index, len(df)):\n    try:\n        df['thumbnail_text'][index] = urlopen(df['thumbnail'][index]).read()\n        print(index)\n    except:\n        df.drop(index, inplace=True)\n        print('drop', index)\n\nprint(len(df))\n"

## 2. collect view count

In [9]:
start_index=len(df)
for index in range(len(df)):
    if(df['view'][index]==-1):
        start_index=index
        break

In [10]:
start_index

8036

In [8]:
for index in range(start_index, len(df)):
    if(df['view'][index]==-1):
        response = youtube.videos().list(
            part = 'statistics',
            id = df.loc[index, 'video_id']
        ).execute()

        if (response['pageInfo']['totalResults'] > 0):
            if('viewCount' in response['items'][0]['statistics']):
                df.loc[index, 'view'] = response['items'][0]['statistics']['viewCount']
            else: 
                df.loc[index, 'view'] = -3
        else:
            df.loc[index, 'view'] = -2

In [16]:
#조회수 조회 불가능한 데이터 삭제
print(len(df))
for index in range(len(df)):
    if(df.loc[index, 'view']<-1):
        df.drop(index, inplace=True)
        print('drop', index)
print(len(df))

8026
drop 7553
8025


## 3. make csv file

In [17]:
df.to_csv('english_data_including_thumbnailtext.csv', index = False)
df

,title,video_id,channel_name,channel_id,thumbnail,publish_time,view,thumbnail_text
0,VLOG: МЫ ТАК ПОДУМАЛИ! 25.05.21,kaibm6LSmb4,Katy LifeVlog,UCpm6vCgiehSKHtMFFS5ukiA,https://i.ytimg.com/vi/kaibm6LSmb4/mqdefault.jpg,2021-05-26T02:30:05Z,101580,"{'TextDetections': [{'DetectedText': 'Sweets',..."
1,RC Car with Train Horns on Golf Course,zYgAf2eu8sQ,Vlog Creations,UCR0VLWitB2xM4q7tjkoJUPw,https://i.ytimg.com/vi/zYgAf2eu8sQ/mqdefault.jpg,2021-05-24T16:00:21Z,1147867,{'TextDetections': [{'DetectedText': 'INSURANC...
2,Brought my FATHER&#39;s PILLOW to sleep | Ss V...,OtACAGFKpn8,Sambhavna Seth Entertainment,UChZ0JmGgyOWSU-1iXxPhsVA,https://i.ytimg.com/vi/OtACAGFKpn8/mqdefault.jpg,2021-05-26T06:25:15Z,429219,"{'TextDetections': [{'DetectedText': 'for', 'T..."
3,6AM PRODUCTIVE online school morning routine/v...,2_q4T1abM8Q,Kristy Bae,UCiLSb-cS_0pkjIYsm9oOB0w,https://i.ytimg.com/vi/2_q4T1abM8Q/mqdefault.jpg,2021-04-29T14:00:02Z,607709,"{'TextDetections': [{'DetectedText': 'I', 'Typ..."
4,My Funniest Vlog Ever 😂 Ayachi Thakur Vlogs,tHD5ajeSxKQ,Ayachi Thakur Vlogs,UC8RXP2epKG3uiPO5QYQyGqg,https://i.ytimg.com/vi/tHD5ajeSxKQ/mqdefault.jpg,2021-05-26T01:30:07Z,46413,"{'TextDetections': [], 'TextModelVersion': '3...."
...,...,...,...,...,...,...,...,...
8021,SI YO FUERA MADRE | Lyna Vlogs,HG6Qkqv5eBQ,Lyna Vlogs,UCRtkuS3Wz0hYgfJjpvEEP4Q,https://i.ytimg.com/vi/HG6Qkqv5eBQ/default.jpg,2017-10-08T22:39:42Z,4884575,not-yet
8022,| DHARAN | Dasain Vlog 2017 | Bhedetar,ZqPTPoyr2M4,Sisan Baniya,UC6SEUWgSEUOTKdwdRV7IRhA,https://i.ytimg.com/vi/ZqPTPoyr2M4/default.jpg,2017-10-09T06:36:39Z,296289,not-yet
8023,VLOG! ALA E TRUMP!,sA3Z_aHJrh8,MaxINFINITE,UCcLZdjaJEC_6RqfM3PhefIw,https://i.ytimg.com/vi/sA3Z_aHJrh8/default.jpg,2017-10-07T08:32:58Z,491894,not-yet
8024,VLOG #78: Hướng dẫn đi máy bay cho người đi lầ...,cwlD15TESUI,Yêu Máy Bay,UC6iYLWyzdYKMpnWbplhBrxg,https://i.ytimg.com/vi/cwlD15TESUI/default.jpg,2017-10-03T10:07:44Z,512585,not-yet
